In [1]:
from azureml.core import Workspace
# from azureml.core.webservice import BatchEndpoint #didn't work
# from azure.ai.ml.entities import BatchEndpoint

from azure.ai.ml.entities import BatchEndpoint, BatchDeployment, Model, AmlCompute, Data, BatchRetrySettings
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction
from azure.identity import DefaultAzureCredential

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (packaging 23.0 (c:\users\1000059312\python\miniconda3\envs\azure-ml\lib\site-packages), Requirement.parse('packaging<22.0,>=20.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (packaging 23.0 (c:\users\1000059312\python\miniconda3\envs\azure-ml\lib\site-packages), Requirement.parse('packaging<22.0,>=20.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (packaging 23.0 (c:\users\1000059312\python\miniconda3\envs\azure-ml\lib\site-packages), Requirement.parse('packaging<22.0,>=20.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun =

In [2]:
from azure.ai.ml import MLClient, Input

subscription_id = "0f644ff4-434a-4446-a414-16ddb2b713ee"
resource_group = "rg-AIML"
workspace = "ws-azureml"

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

In [ ]:
# from dotenv import load_dotenv

# load_dotenv(".env")

# from azure.identity import DefaultAzureCredential

# credential = DefaultAzureCredential()
# # Check if given credential can get token successfully.
# credential.get_token("https://management.azure.com/.default")

In [5]:
try:
    ml_client = MLClient.from_config(credential=DefaultAzureCredential())
except Exception as ex:
    # NOTE: Update following workspace information to contain
    #       your subscription ID, resource group name, and workspace name
    client_config = {
        "subscription_id": "0f644ff4-434a-4446-a414-16ddb2b713ee",
        "resource_group": "rg-AIML",
        "workspace_name": "ws-azureml"
    }

    # write and reload from config file
    import json, os

    config_path = "../.azuremlbatch/config.json"
    os.makedirs(os.path.dirname(config_path), exist_ok=True)
    with open(config_path, "w") as fo:
        fo.write(json.dumps(client_config))
    ml_client = MLClient.from_config(credential=DefaultAzureCredential(), path=config_path)

print(ml_client)

Found the config file in: .\config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x0000021BA0873D90>,
         subscription_id=0f644ff4-434a-4446-a414-16ddb2b713ee,
         resource_group_name=rg-AIML,
         workspace_name=ws-azureml)


In [6]:
compute = [i for i in ml_client.compute.list()]
print(compute)

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: Azure Active Directory error '(invalid_grant) AADSTS50173: The provided grant has expired due to it being revoked, a fresh auth token is needed. The user might have changed or reset their password. The grant was issued on '2023-03-21T12:40:16.8030000Z' and the TokensValidFrom date (before which tokens are not valid) for this user is '2023-04-03T04:43:55.0000000Z'.
Trace ID: b71537fd-842f-4080-bd33-8302c51c5000
Correlation ID: 057877a3-2f79-4af2-ba8c-f9bd5beccf1f
Timestamp: 2023-05-02 06:36:32Z'
Content: {"error":"in

ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: Azure Active Directory error '(invalid_grant) AADSTS50173: The provided grant has expired due to it being revoked, a fresh auth token is needed. The user might have changed or reset their password. The grant was issued on '2023-03-21T12:40:16.8030000Z' and the TokensValidFrom date (before which tokens are not valid) for this user is '2023-04-03T04:43:55.0000000Z'.
Trace ID: b71537fd-842f-4080-bd33-8302c51c5000
Correlation ID: 057877a3-2f79-4af2-ba8c-f9bd5beccf1f
Timestamp: 2023-05-02 06

In [ ]:
deployment_name = "buf-model-1"
endpoint_name = "https://buf-batch-job.centralindia.inference.ml.azure.com/jobs"

ml_client.batch_deployments.get(name=deployment_name, endpoint_name="buf-batch-job")

In [ ]:
ws = Workspace.get(subscription_id='0f644ff4-434a-4446-a414-16ddb2b713ee',
                    resource_group='rg-AIML',
                    name='ws-azureml')
print('\nCompute Targets:')
if ws.compute_targets != {} or len(ws.compute_targets) != 0:
    for compute_name in ws.compute_targets:
        compute = ws.compute_targets[compute_name]
        # compute = ComputeTarget._get(ws, compute_name)
        print("\t",compute.name, ":", compute.type)
else:
    print("\tNone")

In [ ]:
##################################################
##
##      CHATGPT Code                           ##
##
##################################################
# Load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()

# Get the batch endpoint by name
batch_endpoint = BatchEndpoint.get(ws, "my-batch-endpoint")

# Submit a batch job
job = batch_endpoint.submit_job(
input_data="https://myaccount.blob.core.windows.net/mycontainer/myinputdata.csv",
output_data="https://myaccount.blob.core.windows.net/mycontainer/myoutputdata.csv"
)

# Wait for the job to complete
job.wait_for_completion(show_output=True)